### Stick-breaking formulation for ordinal classification

http://proceedings.mlr.press/v22/khan12/khan12.pdf

In [1]:
import numpy as np
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *

In [2]:
k = 10

In [3]:
l_in = InputLayer((None, k-1)) # for a 4-class problem

In [4]:
l_sp = NonlinearityLayer(l_in, nonlinearity=softplus)

In [5]:
l_agg = DenseLayer(l_sp, num_units=k-1, nonlinearity=linear, W=np.tri(k-1,k-1).T.astype("float32"))

In [6]:
l_merge = ElemwiseMergeLayer([l_in, l_agg], merge_function=T.sub)

In [7]:
l_exp = NonlinearityLayer(l_merge, nonlinearity=T.exp)

In [8]:
l_extra = DenseLayer(l_exp, num_units=k, nonlinearity=linear, W=np.eye(k-1,k).astype("float32"))

In [9]:
extra_mat = l_extra.W.get_value()
extra_mat[:,-1] -= 1
l_extra.W.set_value(extra_mat)

In [10]:
extra_bias = l_extra.b.get_value()
extra_bias[-1] = 1
l_extra.b.set_value( extra_bias )
l_extra.b.get_value()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])

-------

In [11]:
tmp1, tmp2 = get_output([l_sp,l_agg], l_in.input_var)
tmp3 = get_output(l_exp, l_in.input_var)
tmp4 = get_output(l_extra, l_in.input_var)

In [13]:
fake_x = np.random.normal(0,3,size=(1,k-1)).astype("float32")
#fake_x = np.asarray([[1.5,0.11,0.215]]).astype("float32")
print tmp1.eval({l_in.input_var: fake_x})
print tmp2.eval({l_in.input_var: fake_x})


[[  7.55007885e-01   2.07538060e+00   3.17754822e+00   5.83505138e-02
    1.14708461e-01   1.11768462e+00   1.56903536e-04   2.55948629e+00
    4.26801817e-01]]
[[  0.75500788   2.83038849   6.0079367    6.06628722   6.18099568
    7.2986803    7.2988372    9.85832349  10.28512531]]


In [14]:
tmp3.eval({l_in.input_var: fake_x})

array([[  5.29993091e-01,   4.11016977e-01,   5.65307754e-02,
          1.39386876e-04,   2.51402643e-04,   1.39193652e-03,
          1.06126070e-07,   6.24014775e-04,   1.81728580e-05]])

In [17]:
tmp4.eval({l_in.input_var: fake_x}).tolist()

[[0.5299930911335945,
  0.41101697656693015,
  0.05653077538130948,
  0.0001393868755976593,
  0.00025140264337855135,
  0.0013919365239889062,
  1.061260699801705e-07,
  0.000624014774778221,
  1.817285795479545e-05,
  3.413711639776995e-05]]

In [16]:
np.sum(tmp4.eval({l_in.input_var: fake_x}))

1.0

----

In [61]:
np.random.normal(0,1,size=(1,4))

array([[ 0.0039312 ,  1.06453777, -0.94312781, -0.99296491]])